### Model Visualization

In [ ]:
# Imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# Set project root
PROJ_ROOT = os.pardir
sys.path.append(PROJ_ROOT)
from src.features.tree import Tree
from src.models.data_manager import DataManager
from src.models.rntn import RNTN

We look at some of the reviews from the test set, to examine how the trained model makes predictions.

#### Example: Positive Sentiment

#### Example: Negative Sentiment

#### Example: Neutral Sentiment

#### Example: No Sentiment Words

In [ ]:
txt_nosentiment = 'Must be seen to be believed.'
'(3 (2 Must) (2 (2 (2 be) (3 (2 seen) (2 (2 to) (2 (2 be) (2 believed))))) (2 .)))'

#### Example: Mixed Sentiments

#### Similar Words: t-SNE Visualization

One way of validating the generated word vectors is to look at how close they are to each other. Since the word-embeddings in the trained model live in high-dimensional space, these have to be mapped to 2-D space for visualization. t-SNE (t-Stochastic Neighbor Embedding) allows for reducing the vector to 2-D space. 

We look at a few randomly chosen words from each class and see how close these are in 2-D space.

In [ ]:
# Load word embeddings from the model
model_name = 'test'

# Load model
rntn_model = RNTN(model_name=model_name)
L, vocab = rntn_model.get_word_embeddings()

In [ ]:
# Choose words from each class
num_words = 5
labels = list(vocab.keys())[:10]

In [ ]:
# Generate 2-d word embeddings
tsne_model = TSNE()
tsne_embeddings = tsne_model.fit_transform(np.transpose(L))
x = tsne_embeddings[:, 0]
y = tsne_embeddings[:, 1]

In [ ]:
# Show the embeddings
plt.figure(figsize=(16, 16)) 
for i in range(10):
    plt.scatter(x[i],y[i])
    plt.annotate(labels[i],
                 xy=(x[i], y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
plt.show()